In [1]:
import numpy as np
from scipy.optimize import *
import networkx as nx 
import random

In [2]:
def tensor(k):
    t = k[0]
    i = 1
    
    while i < len(k) :
        t = np.kron(t,k[i])
        i+=1
    return t

In [3]:
def Graph_to_Hamiltonian(G,n):
    H = np.zeros((2**n), dtype = 'float64')
    Z = np.array([1,-1],dtype = 'float64')
    for i in range(n):
        j = i+1
        while j<n:
            k = [[1,1]]*n
            k = np.array(k,dtype = 'float64')
                
            if G[i][j] !=0:
                k[i] = Z
                k[j] = Z
                H+= tensor(k)*G[i][j] 
                    
            j+=1
        
    return H

In [4]:
class QAOA:
    def __init__(self,depth,H):
        self.p = depth
        self.H = H
        self.n = int(np.log2(len(self.H)))
        
    
        
    def U_gamma(self,angle,state):
        
        t = -1j*angle
        state = state*np.exp(t*self.H.reshape(2**self.n,1))
        
        return state
            
    
    def V_beta(self,angle,state):
        sigmax = np.array([[0,1],[1,0]],dtype = 'float64') 
        for i in range(self.n):
            k = [np.eye(2).astype('float64')]*self.n
            k[i] = sigmax
            state = np.cos(angle)*state + (-1j*np.sin(angle)*np.matmul(tensor(k),state))
            
        return state
    
    
    def qaoa_ansatz(self, angles):
        state = np.ones((2**self.n,1),dtype = 'complex128')*(1/np.sqrt(2**self.n))
        for i in range(self.p):
            state = self.U_gamma(angles[i],state)
            state = self.V_beta(angles[self.p + i],state)
        
        return state 
        
    def expectation(self,angles):
        state = self.qaoa_ansatz(angles)
        
        ex = np.vdot(state,state*(self.H).reshape((2**self.n,1)))
            
        return np.real(ex)
            
    
    def overlap(self,state):

        gener = min(self.H)
        olap = 0
        for i in range(len(self.H)):


            if self.H[i] == gener:
                olap+= np.absolute(state[i])**2

        return olap
    
    
    
    def run(self):
        initial_angles=[]
        bds= [(0.1,2*np.pi)]*self.p + [(0.1,1*np.pi)]*self.p
        for i in range(2*self.p):
            if i < self.p:
                initial_angles.append(random.uniform(0,2*np.pi))
            else:
                initial_angles.append(random.uniform(0,np.pi))
            
        
        
        res = minimize(self.expectation,initial_angles,method='L-BFGS-B', jac=None, bounds=bds, options={'maxiter': 5000})
        
        self.cost = self.expectation(res.x)
        self.f_state = self.qaoa_ansatz(res.x)
        self.olap = self.overlap(self.f_state)[0]
        
        
        
        return self.cost,self.olap
    
    
    
                
        
        
        
        
            
        

        

In [ ]:
# # TEST CODE: Runs 1p-QAOA on MAX-CUT Hamiltonian. Instance generated from a random graph with 6 nodes and 10 edges. 
# nodes = 6
# edges = 10
# depth = 5
# G = nx.gnm_random_graph(nodes,edges)
# G = nx.to_numpy_array(G)
# H = Graph_to_Hamiltonian(G,nodes)
# Q = QAOA(depth,H)
# result = Q.run()
# print(f'QAOA cost:{result[0]} , Success probability: {result[1]}')